<a href="https://colab.research.google.com/github/andyburgin/kubefix-llm/blob/main/k8s_qna_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this notebook

> The objective here is to generate a series of questions and answers from a subset of the kubernetes documentation, the resultant q and a pairs can then be used as a dataset to finetune a LLM for problem solving in [k8sgpt](https://k8sgpt.ai/)

> This notebook is part of the scripts and tools needed to gnerate the dataset over at https://github.com/andyburgin/kubefix-llm



# Prequisites install - models, libraries and data

## Download openchat model

After much testing the openchat model is at the time of writing the best model for generating q and a pairs in json from the Kubernetes website markdown files.

(It also has an apache 2 license to avoid any data generation licensing restrictions unlike a lot of populat models).

In [ ]:
!pip3 install huggingface-hub
!huggingface-cli download TheBloke/openchat-3.5-0106-GGUF openchat-3.5-0106.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/openchat-3.5-0106-GGUF/resolve/main/openchat-3.5-0106.Q5_K_M.gguf to /root/.cache/huggingface/hub/tmpmi96821m
openchat-3.5-0106.Q5_K_M.gguf: 100% 5.13G/5.13G [04:47<00:00, 17.9MB/s]
./openchat-3.5-0106.Q5_K_M.gguf


## Install lamma-cpp-python

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp310-cp310-linux_x86_64.whl size=55266130 sha256=67f058c62671b7e3609a69a6a35cf367de01eeec9fb3ca20eeaf408ab6c856c9
  Stored in directory: /root/.cache/pip/wheels/e4/18/46/58b5c613b17c8d000d79ae650980fe871b3b490e04e6faa1c1
Successfully built llama-cpp-python


## Install LangChain libraries

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


## Clone k8s website

In [ ]:
!git clone https://github.com/kubernetes/website.git

Cloning into 'website'...
remote: Enumerating objects: 403235, done.
remote: Counting objects: 100% (728/728), done.
remote: Compressing objects: 100% (458/458), done.
remote: Total 403235 (delta 437), reused 477 (delta 265), pack-reused 402507
Receiving objects: 100% (403235/403235), 438.54 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (296073/296073), done.
Updating files: 100% (9451/9451), done.


## Create folder structure

In [ ]:
!mkdir out

## IMPORTANT - upload kubedoclist.txt !!!!

This is available over at https://github.com/andyburgin/kubefix-llm it's basically a list of paths to the md files in /content/en/docs folder of the kubernetes website (minus the `.md` extension).

Any lines starting with `#` will be ignored - we don't need to process every doc for our purposes.

In [ ]:
# noop

# Run Q and A generation



## Run model against each file in `/content/website/content/en/docs/` listed in `kubedoclist.txt` not prefixed with`#` creating json output in `out` folder

You're probably gonna hit some problems with some of the bigger markdown files, they might go into an infite generation loop or start producing nonsense - if/when that happens...

* Stop the script, tar and download the files you have (see next step).
* Make a note of the problem file and add it to your `kubedoclist-fix.txt`.
* Comment out the file in your `kubedoclist.txt` file and re upload.
* Run this cell again.


Note that some of the files generated won't be valid json and will require manual fixing up - more below.

In [ ]:
import time
import os
import re
import gc

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
CallbackManager = CallbackManager([StreamingStdOutCallbackHandler()])

from langchain_community.llms import LlamaCpp
from langchain_core.runnables import RunnablePassthrough

start = time.time()
print("Processing started")

# paths to input and output
inputPath = "/content/website/content/en/docs/"
outputPath = "/content/out/"
documentList = "/content/kubedoclist.txt"

# tidy up from previous run in collab
rag_chain = None
llm = None
gc.collect()


def create_llm():
  return LlamaCpp(model_path="/content/openchat-3.5-0106.Q5_K_M.gguf",  n_gpu_layers=128, temperature=0.2, max_tokens=32768, n_ctx=32768, n_batch=8, n_threads=8, verbose=True, callback_manager=CallbackManager)


def create_prompt():
  ## prompt inspired by https://helixml.substack.com/p/how-we-got-fine-tuning-mistral-7b?utm_source=pocket_saves
  sys_prompt = """
  You are an intelligent professor. You create at least 5 question and answer pairs from the given document for your students. Respond with an array of strict JSON 'question' & 'answer' pairs.

  For example:
  ```json
    [
      {{
        "question": "…",
        "answer": "…"
      }},
      {{
        "question": "…",
        "answer": "…"
      }}
    ]
  ```
  """
  instruction = """
  Here is the document:

  {content}
  """

  prompt = "GPT4 Correct User: " + sys_prompt + "\n\n" + instruction + "<|end_of_turn|>GPT4 Correct Assistant:"
  return PromptTemplate.from_template(template=prompt)


def call_model(content):
  response = rag_chain.invoke(content)
  print(response)
  return(response)


# main
fileDocs = open(documentList,"r")
docList = fileDocs.read().splitlines()
fileDocs.close()

custom_rag_prompt = create_prompt()

llm = create_llm()

rag_chain = (
{"content": RunnablePassthrough()}
| custom_rag_prompt
| llm
| StrOutputParser()
)

for filenamepre in docList:
  if not filenamepre.startswith('#') :
    if os.path.exists(outputPath + filenamepre + ".json"):
      print("Skipping doc - file exists " + filenamepre + ".json")
    else:
      print("Processing doc " + filenamepre + ".md")

      # open input file from documentList with appended ".md"
      fileInput = open(inputPath + filenamepre + ".md", "r")
      content = fileInput.read()
      fileInput.close()

      # remove hugo template stuff
      pattern =  r'{{.*?}}'
      content = re.sub(pattern, "", content, flags=re.DOTALL)
      pattern =  r'<!--.*?-->'
      content = re.sub(pattern, "", content, flags=re.DOTALL)

      response = call_model(content)
      response = bytes(response, 'utf-8').decode('unicode_escape')

      # make outupt folder if it doesn't exist
      if not os.path.exists(os.path.dirname(outputPath+filenamepre)):
        os.makedirs(os.path.dirname(outputPath+filenamepre))

      fileOutput = open(outputPath + filenamepre + ".json", "w")
      fileOutput.write(response)
      fileOutput.close()

end = time.time()
print(f"\n\n(took {round(end - start, 2)} s.):")



llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/openchat-3.5-0106.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-0106
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

Processing started


llm_load_vocab: special tokens definition check successful ( 261/32002 ).
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = SPM
llm_load_print_meta: n_vocab          = 32002
llm_load_print_meta: n_merges         = 0
llm_load_print_meta: n_ctx_train      = 8192
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0e-05
llm_load_print_meta: f_clamp_kqv      = 0.0e+00
llm_load_print_meta: f_max_alibi_bias = 0.0e+00
llm_load_print_me

Skipping doc - file exists reference/access-authn-authz/authorization.json
Skipping doc - file exists reference/access-authn-authz/bootstrap-tokens.json
Skipping doc - file exists reference/access-authn-authz/certificate-signing-requests.json
Skipping doc - file exists reference/access-authn-authz/kubelet-authn-authz.json
Skipping doc - file exists reference/access-authn-authz/node.json
Skipping doc - file exists reference/access-authn-authz/psp-to-pod-security-standards.json
Processing doc reference/access-authn-authz/service-accounts-admin.md
 [
  {
    "question": "What is a ServiceAccount in Kubernetes?",
    "answer": "A ServiceAccount provides an identity for processes that run in a Pod. A process inside a Pod can use the identity of its associated service account to authenticate to the cluster's API server."
  },
  {
    "question": "What is the difference between user accounts and service accounts?",
    "answer": "User accounts are for humans, while service accounts are for ap


llama_print_timings:        load time =     192.19 ms
llama_print_timings:      sample time =     714.68 ms /  1214 runs   (    0.59 ms per token,  1698.67 tokens per second)
llama_print_timings: prompt eval time =   80454.50 ms /  6061 tokens (   13.27 ms per token,    75.33 tokens per second)
llama_print_timings:        eval time =   52661.70 ms /  1213 runs   (   43.41 ms per token,    23.03 tokens per second)
llama_print_timings:       total time =  143967.27 ms /  7274 tokens


...

 [
  {
    "question": "What is Validating Admission Policy?",
    "answer": "Validating Admission Policy is a declarative, in-process alternative to validating admission webhooks. It uses the Common Expression Language (CEL) to declare the validation rules of a policy."
  },
  {
    "question": "What resources make a policy?",
    "answer": "A policy is generally made up of three resources: The `ValidatingAdmissionPolicy`, a `ValidatingAdmissionPolicyBinding` and a parameter resource if configuration is needed. At least a `ValidatingAdmissionPolicy` and a corresponding  `ValidatingAdmissionPolicyBinding` must be defined for a policy to have an effect."
  },
  {
    "question": "What are the validation actions?",
    "answer": "The supported validation actions are: `Deny`, `Warn`, `Audit`. For example, to both warn clients about a validation failure and to audit the failure, use: `validationActions: [Warn, Audit]`."
  },
  {
    "question": "What is a parameter resource?",
    "answer"

Llama.generate: prefix-match hit


 [
  {
    "question": "What is a WebHook?",
    "answer": "A WebHook is an HTTP callback: an HTTP POST that occurs when something happens; a simple event-notification via HTTP POST. A web application implementing WebHooks will POST a message to a URL when certain things happen."
  },
  {
    "question": "What does the mode `Webhook` cause Kubernetes to do?",
    "answer": "When specified, mode `Webhook` causes Kubernetes to query an outside REST service when determining user privileges."
  },
  {
    "question": "What is the configuration file format for mode `Webhook`?",
    "answer": "The configuration file uses the [kubeconfig](/docs/tasks/access-application-cluster/configure-access-multiple-clusters/) file format."
  },
  {
    "question": "What does a SubjectAccessReview object describe?",
    "answer": "A SubjectAccessReview object describes the action, user attempting to make the request, and either details about the resource being accessed or request attributes."
  },
  {
    


llama_print_timings:        load time =     192.19 ms
llama_print_timings:      sample time =     200.30 ms /   351 runs   (    0.57 ms per token,  1752.35 tokens per second)
llama_print_timings: prompt eval time =   20261.55 ms /  1608 tokens (   12.60 ms per token,    79.36 tokens per second)
llama_print_timings:        eval time =   11718.42 ms /   350 runs   (   33.48 ms per token,    29.87 tokens per second)
llama_print_timings:       total time =   34501.65 ms /  1958 tokens


 [
  {
    "question": "What is a WebHook?",
    "answer": "A WebHook is an HTTP callback: an HTTP POST that occurs when something happens; a simple event-notification via HTTP POST. A web application implementing WebHooks will POST a message to a URL when certain things happen."
  },
  {
    "question": "What does the mode `Webhook` cause Kubernetes to do?",
    "answer": "When specified, mode `Webhook` causes Kubernetes to query an outside REST service when determining user privileges."
  },
  {
    "question": "What is the configuration file format for mode `Webhook`?",
    "answer": "The configuration file uses the [kubeconfig](/docs/tasks/access-application-cluster/configure-access-multiple-clusters/) file format."
  },
  {
    "question": "What does a SubjectAccessReview object describe?",
    "answer": "A SubjectAccessReview object describes the action, user attempting to make the request, and either details about the resource being accessed or request attributes."
  },
  {
    

## Archive files and download `qanda.tgz`

In [ ]:
!tar -zcvf qanda.tgz /content/out/*


tar: Removing leading `/' from member names
/content/out/concepts/
/content/out/concepts/configuration/
/content/out/concepts/configuration/secret.3.json
/content/out/concepts/configuration/secret.4.json
/content/out/concepts/configuration/secret.2.json
/content/out/concepts/configuration/secret.1.json


# Fixing the failed generations



## Create Q and A in raw text for the docs that the original script just refused to output as json or got confused and started generating endless nonsense

Create a file `kubedoclist-fix.txt` with list of file names to fix and output to json:
```
concepts/cluster-administration/flow-control
concepts/configuration/manage-resources-containers
concepts/extend-kubernetes/compute-storage-net/device-plugins
concepts/security/security-checklist
concepts/security/service-accounts
concepts/services-networking/service
concepts/storage/projected-volumes
concepts/storage/volumes
concepts/storage/persistent-volumes
concepts/storage/storage-classes
concepts/windows/intro
concepts/workloads/controllers/deployment
concepts/workloads/controllers/job
reference/access-authn-authz/admission-controllers
reference/access-authn-authz/authentication
reference/access-authn-authz/certificate-signing-requests
reference/access-authn-authz/extensible-admission-controllers
reference/access-authn-authz/kubelet-tls-bootstrapping
reference/access-authn-authz/rbac
setup/best-practices/certificates
setup/best-practices/cluster-large
tasks/access-application-cluster/ingress-minikube
tasks/administer-cluster/encrypt-data
tasks/administer-cluster/kms-provider
tasks/administer-cluster/memory-manager
tasks/configure-pod-container/configure-service-account
tasks/debug/debug-application/debug-service
tasks/extend-kubernetes/custom-resources/custom-resource-definition-versioning
tasks/extend-kubernetes/custom-resources/custom-resource-definitions
tasks/manage-kubernetes-objects/declarative-config
tasks/manage-kubernetes-objects/kustomization
tasks/run-application/horizontal-pod-autoscale
```
Download each of the source documents and break into `flow-control.md.1`, `flow-control.md.2` and `flow-control.md.3` and place in the fix folder, the below script will produce q and a for the partial files and place them in the `out` folder as a json file, download these and merge into the fixed up json files - sadly you're going to have to fix these up by hand afterwards with the dataset gen script

## generate q and a pairs from fixed documents

In [ ]:
import time
import os
import re
import gc

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
CallbackManager = CallbackManager([StreamingStdOutCallbackHandler()])

from langchain_community.llms import LlamaCpp
from langchain_core.runnables import RunnablePassthrough

start = time.time()
print("Processing started")

# paths to input and output
inputPath = "/content/website/content/en/docs/"
outputPath = "/content/out/"
documentList = "/content/kubedoclist-fix.txt"

# tidy up from previous run in collab
rag_chain = None
llm = None
gc.collect()


def create_llm():
  return LlamaCpp(model_path="/content/openchat-3.5-0106.Q5_K_M.gguf",  n_gpu_layers=128, temperature=0.2, max_tokens=32768, n_ctx=32768, n_batch=8, n_threads=8, verbose=True, callback_manager=CallbackManager)


def create_prompt():
  ## prompt inspired by https://helixml.substack.com/p/how-we-got-fine-tuning-mistral-7b?utm_source=pocket_saves
  sys_prompt = """
  You are an intelligent professor. You create at least 5 question and answer pairs from the given document for your students. Respond with an array of strict JSON 'question' & 'answer' pairs.

  For example:
  ```json
    [
      {{
        "question": "…",
        "answer": "…"
      }},
      {{
        "question": "…",
        "answer": "…"
      }}
    ]
  ```
  """
  instruction = """
  Here is the document:

  {content}
  """

  prompt = "GPT4 Correct User: " + sys_prompt + "\n\n" + instruction + "<|end_of_turn|>GPT4 Correct Assistant:"
  return PromptTemplate.from_template(template=prompt)


def call_model(content):
  response = rag_chain.invoke(content)
  print(response)
  return(response)


# main
fileDocs = open(documentList,"r")
docList = fileDocs.read().splitlines()
fileDocs.close()

custom_rag_prompt = create_prompt()

llm = create_llm()

rag_chain = (
{"content": RunnablePassthrough()}
| custom_rag_prompt
| llm
| StrOutputParser()
)

for filenamepre in docList:
  if not filenamepre.startswith('#') :
    if os.path.exists(outputPath + filenamepre + ".json"):
      print("Skipping doc - file exists " + filenamepre + ".json")
    else:
      print("Processing doc " + filenamepre + ".md")

      # open input file from documentList with appended ".md"
      fileInput = open(inputPath + filenamepre + ".md", "r")
      content = fileInput.read()
      fileInput.close()

      # remove hugo template stuff
      pattern =  r'{{.*?}}'
      content = re.sub(pattern, "", content, flags=re.DOTALL)
      pattern =  r'<!--.*?-->'
      content = re.sub(pattern, "", content, flags=re.DOTALL)

      response = call_model(content)
      response = bytes(response, 'utf-8').decode('unicode_escape')

      #resplines = response.split("\n")
      #response = "\n".join(resplines)

      # make outupt folder if it doesn't exist
      if not os.path.exists(os.path.dirname(outputPath+filenamepre)):
        os.makedirs(os.path.dirname(outputPath+filenamepre))

      fileOutput = open(outputPath + filenamepre + ".json", "w")
      fileOutput.write(response)
      fileOutput.close()

end = time.time()
print(f"\n\n(took {round(end - start, 2)} s.):")

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/openchat-3.5-0106.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-0106
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

Processing started


llm_load_vocab: special tokens definition check successful ( 261/32002 ).
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = SPM
llm_load_print_meta: n_vocab          = 32002
llm_load_print_meta: n_merges         = 0
llm_load_print_meta: n_ctx_train      = 8192
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0e-05
llm_load_print_meta: f_clamp_kqv      = 0.0e+00
llm_load_print_meta: f_max_alibi_bias = 0.0e+00
llm_load_print_me

Processing doc concepts/configuration/secret.1.md
 [
  {
    "question": "What is a Secret in Kubernetes?",
    "answer": "A Secret in Kubernetes is an object that contains sensitive data such as passwords, tokens, or keys. It is used to store and manage confidential information without including it directly in the application code."
  },
  {
    "question": "What are some uses for Secrets?",
    "answer": "Secrets can be used for purposes such as setting environment variables for a container, providing credentials like SSH keys or passwords to Pods, and allowing the kubelet to pull container images from private registries."
  },
  {
    "question": "What are some alternatives to using Secrets?",
    "answer": "Alternatives to using Secrets include ServiceAccount tokens, third-party tools for managing sensitive data, custom signers for X.509 certificates, and device plugins for exposing node-local encryption hardware."
  },
  {
    "question": "What is the default type of Secret in Kub


llama_print_timings:        load time =     316.11 ms
llama_print_timings:      sample time =     264.78 ms /   454 runs   (    0.58 ms per token,  1714.66 tokens per second)
llama_print_timings: prompt eval time =   34362.56 ms /  3008 tokens (   11.42 ms per token,    87.54 tokens per second)
llama_print_timings:        eval time =   15573.85 ms /   454 runs   (   34.30 ms per token,    29.15 tokens per second)
llama_print_timings:       total time =   53780.09 ms /  3462 tokens


...

 [
  {
    "question": "What is a Kubernetes Secret?",
    "answer": "A Kubernetes Secret is an object that contains a small amount of sensitive data such as a password, a token, or a key. It helps to keep confidential data out of your application code and provides additional security measures."
  },
  {
    "question": "What are the default storage locations for Kubernetes Secrets?",
    "answer": "By default, Kubernetes Secrets are stored unencrypted in the API server's underlying data store (etcd). Anyone with API access can retrieve or modify a Secret, and so can anyone with access to etcd."
  },
  {
    "question": "What are some best practices for managing Kubernetes Secrets?",
    "answer": "Some best practices for managing Kubernetes Secrets include enabling encryption at rest, configuring RBAC rules with least-privilege access, restricting Secret access to specific containers, and considering using external Secret store providers."
  },
  {
    "question": "What is a TLS Secre

Llama.generate: prefix-match hit


 [
   {
     "question": "What is a Secret in Kubernetes?",
     "answer": "A Secret is an object that contains a small amount of sensitive data such as a password, a token, or a key."
   },
   {
     "question": "How are Secrets stored by default in Kubernetes?",
     "answer": "By default, Secrets are stored unencrypted in the API server's underlying data store (etcd)."
   },
   {
     "question": "What is one way to access data from a Secret in a Pod?",
     "answer": "One way to access data from a Secret in a Pod is to have Kubernetes make the value of that Secret be available as a file inside the filesystem of one or more of the Pod's containers."
   },
   {
     "question": "What is an immutable Secret?",
     "answer": "An immutable Secret is a Secret that has its data field marked as read-only, preventing accidental updates and improving cluster performance."
   },
   {
     "question": "How can you configure least-privilege access to Secrets in Kubernetes?",
     "answer": "Yo


llama_print_timings:        load time =     316.11 ms
llama_print_timings:      sample time =     182.65 ms /   318 runs   (    0.57 ms per token,  1741.02 tokens per second)
llama_print_timings: prompt eval time =   24004.24 ms /  1927 tokens (   12.46 ms per token,    80.28 tokens per second)
llama_print_timings:        eval time =   11007.40 ms /   317 runs   (   34.72 ms per token,    28.80 tokens per second)
llama_print_timings:       total time =   37437.07 ms /  2244 tokens


 [
   {
     "question": "What is a Secret in Kubernetes?",
     "answer": "A Secret is an object that contains a small amount of sensitive data such as a password, a token, or a key."
   },
   {
     "question": "How are Secrets stored by default in Kubernetes?",
     "answer": "By default, Secrets are stored unencrypted in the API server's underlying data store (etcd)."
   },
   {
     "question": "What is one way to access data from a Secret in a Pod?",
     "answer": "One way to access data from a Secret in a Pod is to have Kubernetes make the value of that Secret be available as a file inside the filesystem of one or more of the Pod's containers."
   },
   {
     "question": "What is an immutable Secret?",
     "answer": "An immutable Secret is a Secret that has its data field marked as read-only, preventing accidental updates and improving cluster performance."
   },
   {
     "question": "How can you configure least-privilege access to Secrets in Kubernetes?",
     "answer": "Yo

#Finally

OK so you've managed to get all of the markdown files converted into valid json - you'll now appreciate the amount of manual work needed to get this working.

Next step is to finetune the model, head back to https://github.com/andyburgin/kubefix-llm and see what's next